In [39]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import mxnet as mx
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import DeepAREstimator, Trainer, MQRNNEstimator, TemporalFusionTransformerEstimator
# from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer.callback import TrainingHistory


from gluonts.dataset.util import to_pandas
from gluonts.dataset.pandas import PandasDataset
from gluonts.evaluation.backtest import make_evaluation_predictions

In [40]:
mx.context.num_gpus()

0

In [41]:
df = pd.read_csv('../../../data/Merged-update_hourly.csv')
# df = pd.read_csv('pollution.csv')

In [42]:
df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN
0,1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0
1,1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0
2,1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0
3,1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0
4,1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,2.90,2.21,0.0,253.27,0.0,2.62,2.58,2.180,0.0,0.0


In [43]:
df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'Time'], infer_datetime_format=True)

In [44]:
df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,...,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN,timestamp
0,1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,...,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0,2010-01-01 00:00:00
1,1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,...,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0,2010-01-01 01:00:00
2,1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,...,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0,2010-01-01 02:00:00
3,1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,...,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0,2010-01-01 03:00:00
4,1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,...,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0,2010-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,...,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0,2020-12-31 19:00:00
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,...,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0,2020-12-31 20:00:00
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,...,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0,2020-12-31 21:00:00
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,...,2.21,0.0,253.27,0.0,2.62,2.58,2.180,0.0,0.0,2020-12-31 22:00:00


In [45]:
df.columns

Index(['Time', 'WS_S1', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'TWS_S25A', 'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'TWS_S25B', 'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'TWS_S26', 'PUMP_S26', 'MEAN_RAIN', 'timestamp'],
      dtype='object')

In [46]:
train_df = df.iloc[:int(len(df)*0.8), :]
test_df = df.iloc[int(len(df)*0.8):, :]

In [47]:
train_df.shape

(77145, 21)

In [48]:
test_df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,...,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN,timestamp
77145,10/20/18 09:00,1.06,1.01,0.0,0.0,2.41,1.07,0.0,0.0,0.0,...,1.15,0.0,469.19,0.0,2.61,2.35,1.11,0.0,0.0,2018-10-20 09:00:00
77146,10/20/18 10:00,0.68,0.63,0.0,0.0,2.41,0.71,0.0,0.0,0.0,...,0.82,0.0,0.00,0.0,0.00,2.62,0.72,0.0,0.0,2018-10-20 10:00:00
77147,10/20/18 11:00,0.31,0.32,0.0,0.0,2.41,0.33,0.0,0.0,0.0,...,0.42,0.0,0.00,0.0,0.00,2.62,0.37,0.0,0.0,2018-10-20 11:00:00
77148,10/20/18 12:00,0.20,0.23,0.0,0.0,2.40,0.24,0.0,0.0,0.0,...,0.35,0.0,0.00,0.0,0.00,2.67,0.28,0.0,0.0,2018-10-20 12:00:00
77149,10/20/18 13:00,0.41,0.42,0.0,0.0,2.39,0.42,0.0,0.0,0.0,...,0.48,0.0,0.00,0.0,0.00,2.72,0.45,0.0,0.0,2018-10-20 13:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,...,1.31,0.0,456.34,0.0,2.61,2.49,1.29,0.0,0.0,2020-12-31 19:00:00
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,...,1.68,0.0,380.19,0.0,2.62,2.48,1.66,0.0,0.0,2020-12-31 20:00:00
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,...,2.04,0.0,287.12,0.0,2.62,2.54,2.04,0.0,0.0,2020-12-31 21:00:00
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,...,2.21,0.0,253.27,0.0,2.62,2.58,2.18,0.0,0.0,2020-12-31 22:00:00


In [49]:
train_ds = PandasDataset(
    train_df,
    target = ['WS_S1'],
    timestamp = 'timestamp',
    feat_dynamic_real = ['TWS_S25A', 'TWS_S25B', 'TWS_S26', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
    freq = 'H'
)

In [50]:
test_ds = PandasDataset(
    test_df,
    target = ['WS_S1'],
    timestamp = 'timestamp',
    feat_dynamic_real = ['TWS_S25A', 'TWS_S25B', 'TWS_S26', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
    freq = 'H'
)

#### training

In [51]:
K = 48
n_hours = 24*3

In [52]:
trainer=Trainer(
    learning_rate=1e-3,
    epochs=150,
    num_batches_per_epoch=100,
    add_default_callbacks=True,
    weight_decay=1e-06)

estimator = TemporalFusionTransformerEstimator(
    prediction_length=K,
    freq="H",
#     use_feat_dynamic_real=True,
    batch_size=32,
    trainer=trainer,
    dropout_rate=0.3,
    hidden_dim=64,
    num_heads=1,
#     time_features=['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv'],
)

model = estimator.train(train_ds)

100%|███████████████████████████████████████████████| 100/100 [05:40<00:00,  3.41s/it, epoch=131/150, avg_epoch_loss=0.0278]


100%|███████████████████████████████████████████████| 100/100 [13:48<00:00,  8.29s/it, epoch=150/150, avg_epoch_loss=0.0276]


#### predicted y

In [53]:
quantiles = ['0.5', '0.1', '0.9']
q50 = 0

In [54]:
test_ds_predict_q50_all = np.array([])

for i in range(len(test_df)-n_hours-K+1):
    print(i)
    
    cut = df.iloc[len(train_df)+i:len(train_df)+i+n_hours+K, :]

    test_ds = PandasDataset(
    cut,
    target = 'WS_S1',
    timestamp = 'timestamp',
    feat_dynamic_real = ['TWS_S25A', 'TWS_S25B', 'TWS_S26', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
        freq = 'H'
    )
    
    forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=200)
    test_ds_predict = list(forecast_it)
    test_ds_predict_quantiles = test_ds_predict[0].forecast_array
    test_ds_predict_q50 = test_ds_predict_quantiles.mean(axis=0)
    test_ds_predict_q50_all = np.concatenate((test_ds_predict_q50_all, test_ds_predict_q50), axis=None)
    print(test_ds_predict_q50_all)
    print(test_ds_predict_q50_all.shape)
    print('------')

0
[ 2.50714278  1.9593159   1.43155968  0.93871492  0.49638224  0.30508277
  0.48734465  1.00778925  1.59129238  2.0646503   2.47305655  2.65355682
  2.48545074  1.95148647  1.41619742  0.90187424  0.37553588  0.0727419
  0.15381567  0.63283414  1.29550707  1.83602107  2.34205842  2.69129872
  2.73322105  2.36796546  1.78594244  1.28967178  0.80456418  0.40743789
  0.30151242  0.57764429  1.15362847  1.70449221  2.16608286  2.5255971
  2.61114311  2.30059624  1.73017597  1.2180897   0.68034577  0.19672604
 -0.00815082  0.17484532  0.72361857  1.36863995  1.90979517  2.40809464]
(48,)
------
1
[ 2.50714278e+00  1.95931590e+00  1.43155968e+00  9.38714921e-01
  4.96382236e-01  3.05082768e-01  4.87344652e-01  1.00778925e+00
  1.59129238e+00  2.06465030e+00  2.47305655e+00  2.65355682e+00
  2.48545074e+00  1.95148647e+00  1.41619742e+00  9.01874244e-01
  3.75535876e-01  7.27419034e-02  1.53815672e-01  6.32834136e-01
  1.29550707e+00  1.83602107e+00  2.34205842e+00  2.69129872e+00
  2.733221

[ 2.50714278e+00  1.95931590e+00  1.43155968e+00  9.38714921e-01
  4.96382236e-01  3.05082768e-01  4.87344652e-01  1.00778925e+00
  1.59129238e+00  2.06465030e+00  2.47305655e+00  2.65355682e+00
  2.48545074e+00  1.95148647e+00  1.41619742e+00  9.01874244e-01
  3.75535876e-01  7.27419034e-02  1.53815672e-01  6.32834136e-01
  1.29550707e+00  1.83602107e+00  2.34205842e+00  2.69129872e+00
  2.73322105e+00  2.36796546e+00  1.78594244e+00  1.28967178e+00
  8.04564178e-01  4.07437891e-01  3.01512420e-01  5.77644289e-01
  1.15362847e+00  1.70449221e+00  2.16608286e+00  2.52559710e+00
  2.61114311e+00  2.30059624e+00  1.73017597e+00  1.21808970e+00
  6.80345774e-01  1.96726039e-01 -8.15081596e-03  1.74845323e-01
  7.23618567e-01  1.36863995e+00  1.90979517e+00  2.40809464e+00
  1.96347082e+00  1.44826114e+00  9.62969065e-01  5.27278662e-01
  3.23141813e-01  4.97448534e-01  1.02153802e+00  1.60495055e+00
  2.07542992e+00  2.48106265e+00  2.66328025e+00  2.49914289e+00
  1.96742499e+00  1.42805

[ 2.50714278e+00  1.95931590e+00  1.43155968e+00  9.38714921e-01
  4.96382236e-01  3.05082768e-01  4.87344652e-01  1.00778925e+00
  1.59129238e+00  2.06465030e+00  2.47305655e+00  2.65355682e+00
  2.48545074e+00  1.95148647e+00  1.41619742e+00  9.01874244e-01
  3.75535876e-01  7.27419034e-02  1.53815672e-01  6.32834136e-01
  1.29550707e+00  1.83602107e+00  2.34205842e+00  2.69129872e+00
  2.73322105e+00  2.36796546e+00  1.78594244e+00  1.28967178e+00
  8.04564178e-01  4.07437891e-01  3.01512420e-01  5.77644289e-01
  1.15362847e+00  1.70449221e+00  2.16608286e+00  2.52559710e+00
  2.61114311e+00  2.30059624e+00  1.73017597e+00  1.21808970e+00
  6.80345774e-01  1.96726039e-01 -8.15081596e-03  1.74845323e-01
  7.23618567e-01  1.36863995e+00  1.90979517e+00  2.40809464e+00
  1.96347082e+00  1.44826114e+00  9.62969065e-01  5.27278662e-01
  3.23141813e-01  4.97448534e-01  1.02153802e+00  1.60495055e+00
  2.07542992e+00  2.48106265e+00  2.66328025e+00  2.49914289e+00
  1.96742499e+00  1.42805

[ 2.50714278e+00  1.95931590e+00  1.43155968e+00  9.38714921e-01
  4.96382236e-01  3.05082768e-01  4.87344652e-01  1.00778925e+00
  1.59129238e+00  2.06465030e+00  2.47305655e+00  2.65355682e+00
  2.48545074e+00  1.95148647e+00  1.41619742e+00  9.01874244e-01
  3.75535876e-01  7.27419034e-02  1.53815672e-01  6.32834136e-01
  1.29550707e+00  1.83602107e+00  2.34205842e+00  2.69129872e+00
  2.73322105e+00  2.36796546e+00  1.78594244e+00  1.28967178e+00
  8.04564178e-01  4.07437891e-01  3.01512420e-01  5.77644289e-01
  1.15362847e+00  1.70449221e+00  2.16608286e+00  2.52559710e+00
  2.61114311e+00  2.30059624e+00  1.73017597e+00  1.21808970e+00
  6.80345774e-01  1.96726039e-01 -8.15081596e-03  1.74845323e-01
  7.23618567e-01  1.36863995e+00  1.90979517e+00  2.40809464e+00
  1.96347082e+00  1.44826114e+00  9.62969065e-01  5.27278662e-01
  3.23141813e-01  4.97448534e-01  1.02153802e+00  1.60495055e+00
  2.07542992e+00  2.48106265e+00  2.66328025e+00  2.49914289e+00
  1.96742499e+00  1.42805

[ 2.50714278e+00  1.95931590e+00  1.43155968e+00  9.38714921e-01
  4.96382236e-01  3.05082768e-01  4.87344652e-01  1.00778925e+00
  1.59129238e+00  2.06465030e+00  2.47305655e+00  2.65355682e+00
  2.48545074e+00  1.95148647e+00  1.41619742e+00  9.01874244e-01
  3.75535876e-01  7.27419034e-02  1.53815672e-01  6.32834136e-01
  1.29550707e+00  1.83602107e+00  2.34205842e+00  2.69129872e+00
  2.73322105e+00  2.36796546e+00  1.78594244e+00  1.28967178e+00
  8.04564178e-01  4.07437891e-01  3.01512420e-01  5.77644289e-01
  1.15362847e+00  1.70449221e+00  2.16608286e+00  2.52559710e+00
  2.61114311e+00  2.30059624e+00  1.73017597e+00  1.21808970e+00
  6.80345774e-01  1.96726039e-01 -8.15081596e-03  1.74845323e-01
  7.23618567e-01  1.36863995e+00  1.90979517e+00  2.40809464e+00
  1.96347082e+00  1.44826114e+00  9.62969065e-01  5.27278662e-01
  3.23141813e-01  4.97448534e-01  1.02153802e+00  1.60495055e+00
  2.07542992e+00  2.48106265e+00  2.66328025e+00  2.49914289e+00
  1.96742499e+00  1.42805

...

[2.50714278 1.9593159  1.43155968 ... 2.81624889 2.98354435 2.83924651]
(413280,)
------
8610
[2.50714278 1.9593159  1.43155968 ... 2.99026799 2.85221314 2.46445584]
(413328,)
------
8611
[2.50714278 1.9593159  1.43155968 ... 2.85522103 2.46995783 2.03006196]
(413376,)
------
8612
[2.50714278 1.9593159  1.43155968 ... 2.44831848 2.01157212 1.594419  ]
(413424,)
------
8613
[2.50714278 1.9593159  1.43155968 ... 1.99960101 1.62257421 1.25613809]
(413472,)
------
8614
[2.50714278 1.9593159  1.43155968 ... 1.65967834 1.30651331 1.0727948 ]
(413520,)
------
8615
[2.50714278 1.9593159  1.43155968 ... 1.36068356 1.13137043 1.12443542]
(413568,)
------
8616
[2.50714278 1.9593159  1.43155968 ... 1.12817776 1.11710536 1.37015474]
(413616,)
------
8617
[2.50714278 1.9593159  1.43155968 ... 1.18460763 1.43239844 1.87592125]
(413664,)
------
8618
[2.50714278 1.9593159  1.43155968 ... 1.46676445 1.90482378 2.31496716]
(413712,)
------
8619
[2.50714278 1.9593159  1.43155968 ... 1.88975513 2.30650878 

[2.50714278 1.9593159  1.43155968 ... 2.59497046 2.18485856 1.81929493]
(417600,)
------
8700
[2.50714278 1.9593159  1.43155968 ... 2.20346332 1.84597504 1.48121703]
(417648,)
------
8701
[2.50714278 1.9593159  1.43155968 ... 1.81385171 1.44055939 1.1492573 ]
(417696,)
------
8702
[2.50714278 1.9593159  1.43155968 ... 1.44684637 1.14803874 1.07428563]
(417744,)
------
8703
[2.50714278 1.9593159  1.43155968 ... 1.12437093 1.06672192 1.29191542]
(417792,)
------
8704
[2.50714278 1.9593159  1.43155968 ... 1.05613184 1.27466309 1.64576519]
(417840,)
------
8705
[2.50714278 1.9593159  1.43155968 ... 1.27577305 1.64624214 2.03782082]
(417888,)
------
8706
[2.50714278 1.9593159  1.43155968 ... 1.65453815 2.04752707 2.40030575]
(417936,)
------
8707
[2.50714278 1.9593159  1.43155968 ... 2.00514674 2.35515475 2.64386845]
(417984,)
------
8708
[2.50714278 1.9593159  1.43155968 ... 2.33095312 2.62439942 2.71966839]
(418032,)
------
8709
[2.50714278 1.9593159  1.43155968 ... 2.62917972 2.73191571 

[2.50714278 1.9593159  1.43155968 ... 2.50289798 1.86179829 1.28581953]
(421824,)
------
8788
[2.50714278 1.9593159  1.43155968 ... 1.84497488 1.27869928 0.73776269]
(421872,)
------
8789
[2.50714278 1.9593159  1.43155968 ... 1.29415524 0.75731754 0.4357892 ]
(421920,)
------
8790
[2.50714278 1.9593159  1.43155968 ... 0.75024301 0.43221521 0.51577133]
(421968,)
------
8791
[2.50714278 1.9593159  1.43155968 ... 0.42375317 0.50705951 1.01187861]
(422016,)
------
8792
[2.50714278 1.9593159  1.43155968 ... 0.55842012 1.05411589 1.72875345]
(422064,)
------
8793
[2.50714278 1.9593159  1.43155968 ... 1.04192173 1.71390378 2.31227374]
(422112,)
------
8794
[2.50714278 1.9593159  1.43155968 ... 1.6999644  2.30731678 2.83151317]
(422160,)
------
8795
[2.50714278 1.9593159  1.43155968 ... 2.30274487 2.81870961 3.10823131]
(422208,)
------
8796
[2.50714278 1.9593159  1.43155968 ... 2.78599739 3.07272077 2.99966049]
(422256,)
------
8797
[2.50714278 1.9593159  1.43155968 ... 3.09162354 3.01193523 

[ 2.50714278  1.9593159   1.43155968 ...  0.07561052 -0.17740421
  0.02626785]
(426144,)
------
8878
[ 2.50714278  1.9593159   1.43155968 ... -0.20142198  0.02824287
  0.67601329]
(426192,)
------
8879
[2.50714278 1.9593159  1.43155968 ... 0.01601664 0.63355052 1.43186033]
(426240,)
------
8880
[2.50714278 1.9593159  1.43155968 ... 0.57802033 1.36639547 2.07165909]
(426288,)
------
8881
[2.50714278 1.9593159  1.43155968 ... 1.38946927 2.10029483 2.735955  ]
(426336,)
------
8882
[2.50714278 1.9593159  1.43155968 ... 2.13490105 2.7735908  3.18410492]
(426384,)
------
8883
[2.50714278 1.9593159  1.43155968 ... 2.74259114 3.1564486  3.15369439]
(426432,)
------
8884
[2.50714278 1.9593159  1.43155968 ... 3.14950633 3.14141393 2.62740827]
(426480,)
------
8885
[2.50714278 1.9593159  1.43155968 ... 3.12405849 2.6073792  1.87948227]
(426528,)
------
8886
[2.50714278 1.9593159  1.43155968 ... 2.58354616 1.85037863 1.19021904]
(426576,)
------
8887
[2.50714278 1.9593159  1.43155968 ... 1.837053

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [55]:
test_ds_predict_q50_supervised = test_ds_predict_q50_all.reshape((-1, K))
test_ds_predict_q50_supervised.shape

(19168, 48)

In [56]:
test_ds_predict_q50_supervised[0:3]

array([[ 2.50714278e+00,  1.95931590e+00,  1.43155968e+00,
         9.38714921e-01,  4.96382236e-01,  3.05082768e-01,
         4.87344652e-01,  1.00778925e+00,  1.59129238e+00,
         2.06465030e+00,  2.47305655e+00,  2.65355682e+00,
         2.48545074e+00,  1.95148647e+00,  1.41619742e+00,
         9.01874244e-01,  3.75535876e-01,  7.27419034e-02,
         1.53815672e-01,  6.32834136e-01,  1.29550707e+00,
         1.83602107e+00,  2.34205842e+00,  2.69129872e+00,
         2.73322105e+00,  2.36796546e+00,  1.78594244e+00,
         1.28967178e+00,  8.04564178e-01,  4.07437891e-01,
         3.01512420e-01,  5.77644289e-01,  1.15362847e+00,
         1.70449221e+00,  2.16608286e+00,  2.52559710e+00,
         2.61114311e+00,  2.30059624e+00,  1.73017597e+00,
         1.21808970e+00,  6.80345774e-01,  1.96726039e-01,
        -8.15081596e-03,  1.74845323e-01,  7.23618567e-01,
         1.36863995e+00,  1.90979517e+00,  2.40809464e+00],
       [ 1.96347082e+00,  1.44826114e+00,  9.62969065e-

#### actual y

In [57]:
from helper import series_to_supervised, stage_series_to_supervised

In [58]:
test_df_supervised = series_to_supervised(test_df[['WS_S1']], n_hours, K)

In [59]:
test_df_supervised.iloc[:, -K:].shape

(19168, 48)

In [60]:
test_df_supervised_y = test_df_supervised.iloc[:, -K:]

In [61]:
test_df_supervised_y

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9),...,var1(t+38),var1(t+39),var1(t+40),var1(t+41),var1(t+42),var1(t+43),var1(t+44),var1(t+45),var1(t+46),var1(t+47)
77217,2.53,1.92,1.43,0.92,0.50,0.20,0.47,0.98,1.62,2.12,...,1.62,1.06,0.46,-0.07,-0.16,0.10,0.68,1.36,1.98,2.48
77218,1.92,1.43,0.92,0.50,0.20,0.47,0.98,1.62,2.12,2.53,...,1.06,0.46,-0.07,-0.16,0.10,0.68,1.36,1.98,2.48,2.80
77219,1.43,0.92,0.50,0.20,0.47,0.98,1.62,2.12,2.53,2.75,...,0.46,-0.07,-0.16,0.10,0.68,1.36,1.98,2.48,2.80,2.70
77220,0.92,0.50,0.20,0.47,0.98,1.62,2.12,2.53,2.75,2.59,...,-0.07,-0.16,0.10,0.68,1.36,1.98,2.48,2.80,2.70,2.19
77221,0.50,0.20,0.47,0.98,1.62,2.12,2.53,2.75,2.59,2.05,...,-0.16,0.10,0.68,1.36,1.98,2.48,2.80,2.70,2.19,1.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96380,1.95,1.96,1.62,1.14,0.73,0.15,-0.20,-0.28,-0.05,0.52,...,2.36,2.18,1.69,1.14,0.64,0.11,0.04,0.18,0.77,1.24
96381,1.96,1.62,1.14,0.73,0.15,-0.20,-0.28,-0.05,0.52,1.13,...,2.18,1.69,1.14,0.64,0.11,0.04,0.18,0.77,1.24,1.62
96382,1.62,1.14,0.73,0.15,-0.20,-0.28,-0.05,0.52,1.13,1.59,...,1.69,1.14,0.64,0.11,0.04,0.18,0.77,1.24,1.62,1.98
96383,1.14,0.73,0.15,-0.20,-0.28,-0.05,0.52,1.13,1.59,1.99,...,1.14,0.64,0.11,0.04,0.18,0.77,1.24,1.62,1.98,2.13


#### performance on test set

In [62]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


print('MAE = {}'.format(float("{:.3f}".format(mae(test_ds_predict_q50_supervised, test_df_supervised_y)))))
print('RMSE = {}'.format(float("{:.3f}".format(np.sqrt(mse(test_ds_predict_q50_supervised, test_df_supervised_y))))))

MAE = 0.144
RMSE = 0.194
